CIFAR-100 SIMPLE CNN EXAMPLE

To use this notebook, first download CIFAR100 dataset for Python from https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz and unpack it so that `cifar-100-python` dir is in the same dir as this notebook.

In [4]:
import keras
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [54]:
# here are some helper functions to handle data
def get_proper_images(raw):
    raw_float = np.array(raw, dtype=float) 
    images = raw_float.reshape([-1, 3, 32, 32])
    images = images.transpose([0, 2, 3, 1])
    return images

def onehot_labels(labels):
    return np.eye(100)[labels]

def unpickle(file, decode=True):
    fo = open(file, 'rb')
    raw_dict = pickle.load(fo, encoding='bytes')
    fo.close()
    if decode:
        decoded_dict = {k.decode('utf8'): v for k, v in raw_dict.items()}
    else:
        decoded_dict = raw_dict
    return decoded_dict

def hide_test_labels(file):
    testing_data = unpickle('cifar-100-python/test')
    cleaned_testing_data = ['NIEMA' for elem in testing_data['fine_labels']]
    testing_data['fine_labels'] = cleaned_testing_data
    coded_dict = {k.encode('utf8'): v for k, v in testing_data.items()}
    fo = open(file, 'wb')
    pickle.dump(testing_data, file=fo)
    fo.close()

In [6]:
# here we load the training/testing data
x_train = get_proper_images(unpickle('cifar-100-python/train')['data'])
y_train = onehot_labels(unpickle('cifar-100-python/train')['fine_labels'])
x_test = get_proper_images(unpickle('cifar-100-python/test')['data'])
y_test = onehot_labels(unpickle('cifar-100-python/test')['fine_labels'])


In [7]:
# here are some constants
batch_size = 32
num_classes = 100
epochs = 1
num_predictions = 20
model_name = 'keras_simple_cnn_cifar100.h5'

In [8]:
# here we define the actual model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [9]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [10]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [12]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=False)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 389s 8ms/step - loss: 4.3254 - acc: 0.0467 - val_loss: 3.9075 - val_acc: 0.1183


In [14]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)


In [15]:
model_path = os.path.join(save_dir, model_name)
ath = os.path.join(save_dir, model_name)
print('Saved trained model at %s ' % model_path)

Saved trained model at /home/kuba/Development/saved_models/keras_simple_cnn_cifar100.h5 


In [16]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 18s 2ms/step
Test loss: 3.90754724045
Test accuracy: 0.1183


In [17]:
predictions = model.predict(x_test, verbose=1)

10000/10000 [==============================] - 20s 2ms/step


In [18]:
# prepare data for sample submission
y_test_filenames = unpickle('cifar-100-python/test')['filenames']
y_test_labels = onehot_labels(unpickle('cifar-100-python/test')['fine_labels'])


model_predictions = [np.argmax(predictions, axis=1)
                     for _ in range(len(predictions))]
correct_predictions = [np.argmax(y_test, axis=1)
                      for _ in range(len(y_test))]
correct_predictions_set  = [np.argmax(y_test_labels, axis=1)
                          for _ in range(len(y_test_labels))]


In [66]:
import pandas as pd
submission_df = pd.DataFrame({'filename': [elem.decode('utf-8') for elem in y_test_filenames],
                             'correct_class': correct_predictions[0],
                             'predicted_class': model_predictions[0]})

In [67]:
submission_df.head(5)

,correct_class,filename,predicted_class
0,49,volcano_s_000012.png,23
1,33,woods_s_000412.png,97
2,72,seal_s_001803.png,52
3,51,mushroom_s_001755.png,54
4,71,adriatic_sea_s_000653.png,54


In [68]:
sample_submission = submission_df.drop('correct_class', axis=1)
sample_submission = sample_submission.reindex(columns=['filename', 'predicted_class'])
sample_submission.to_csv('sample_submission.csv', index=False)
sample_submission.head(5)

,filename,predicted_class
0,volcano_s_000012.png,23
1,woods_s_000412.png,97
2,seal_s_001803.png,52
3,mushroom_s_001755.png,54
4,adriatic_sea_s_000653.png,54


In [69]:
solution_file = submission_df.drop('predicted_class', axis=1)
solution_file = solution_file.reindex(columns=['filename', 'correct_class'])
solution_file.to_csv('solution_file.csv', index=False)
solution_file.head()

,filename,correct_class
0,volcano_s_000012.png,49
1,woods_s_000412.png,33
2,seal_s_001803.png,72
3,mushroom_s_001755.png,51
4,adriatic_sea_s_000653.png,71


In [55]:
hide_test_labels('cifar-100-python/test_cleaned')

In [57]:
#test the stuff i've done
test = unpickle('cifar-100-python/test_cleaned', decode=False)['data']
test

array([[199, 196, 195, ..., 215, 211, 183],
       [113,  88,  72, ...,  96,  80,  67],
       [ 61,  61,  67, ..., 189, 115,  89],
       ..., 
       [ 24,  24,  22, ...,  20,  54,  54],
       [ 86,  93,  28, ..., 119, 115, 109],
       [246, 240, 214, ..., 114, 141, 118]], dtype=uint8)